In [1]:
import sys
sys.path.append('/home/cara/Documents/reddit_analysis_code')
import numpy as np
import pandas as pd
from reddit_dataclass import RedditData as reddit
import pickle

In [32]:
dirpath = '/home/cara/Documents/subreddit_data'
filepaths = {
    'books': f'{dirpath}/books_oct2022.csv',
    'conspiracy': f'{dirpath}/conspiracy_oct2022.csv',
    'crypto': f'{dirpath}/cryptocurrency_oct2022.csv',
    'thedonaldmay': f'{dirpath}/thedonald_may2019.csv',
    'thedonaldjune': f'{dirpath}/thedonald_jun2019.csv'
}

In [33]:
datasets = {}
for key in filepaths:
    datasets[key] = reddit(filepaths[key], name=key)

/home/cara/Documents/reddit_analysis_code/reddit_dataclass.py:56: DtypeWarning: Columns (4,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  self.data = self.read_reddit_data(


In [34]:
for key in datasets:
    print(f'\n*** {key} ***\n')
    print(datasets[key].info())


*** books ***

Rows: 111052
Comments: 105892
Posts: 5160
Start time: 2022-10-01 00:00:43
End time: 2022-10-30 23:59:48
Total time: 29 days 23:59:05
Unique authors: 50461
Unique domains: 261
Bot rows: 108734
[deleted] and [removed] rows: 4665

*** conspiracy ***

Rows: 455437
Comments: 444001
Posts: 11436
Start time: 2022-10-01 00:00:02
End time: 2022-10-30 23:59:58
Total time: 29 days 23:59:56
Unique authors: 46447
Unique domains: 833
Bot rows: 462932
[deleted] and [removed] rows: 20209

*** crypto ***

Rows: 476948
Comments: 462106
Posts: 14842
Start time: 2022-10-01 00:00:04
End time: 2022-10-30 23:59:50
Total time: 29 days 23:59:46
Unique authors: 34443
Unique domains: 1270
Bot rows: 478734
[deleted] and [removed] rows: 26217

*** thedonaldmay ***

Rows: 682
Comments: 403
Posts: 279
Start time: 2019-05-01 01:16:01
End time: 2019-05-30 23:58:40
Total time: 29 days 22:42:39
Unique authors: 337
Unique domains: 27
Bot rows: 644
[deleted] and [removed] rows: 33

*** thedonaldjune ***

R

In [36]:
datasets['thedonald'] = reddit(dataframe=pd.concat((datasets['thedonaldmay'].data, datasets['thedonaldjune'].data)), name='thedonald')

In [37]:
datasets.pop('thedonaldmay')
datasets.pop('thedonaldjune')
datasets

{'books': <reddit_dataclass.RedditData at 0x7f9ddaa98a60>,
 'conspiracy': <reddit_dataclass.RedditData at 0x7f9dda1b6c70>,
 'crypto': <reddit_dataclass.RedditData at 0x7f9dda1b6970>,
 'thedonald': <reddit_dataclass.RedditData at 0x7f9db68bb4c0>}

In [84]:
#pickle.dump(datasets, open('datasets.p', 'wb'))
datasets = pickle.load(open('datasets.p', 'rb'))

In [4]:
datasets

{'books': <reddit_dataclass.RedditData at 0x7fb1f7b55430>,
 'conspiracy': <reddit_dataclass.RedditData at 0x7fb1ed8faa90>,
 'crypto': <reddit_dataclass.RedditData at 0x7fb1d0b702e0>,
 'thedonald': <reddit_dataclass.RedditData at 0x7fb1b7810d60>}

In [13]:
summaries = {}
for key in datasets:
    summaries[key] = datasets[key].summary()

info_df = {}
for key in summaries:
    if isinstance(info_df, dict):
        info_df = summaries[key]
    else:
        info_df = pd.concat((info_df, summaries[key]))

info_df.index = summaries.keys()
    
info_df

,rows,comments,posts,start_time,end_time,total_time,unique_authors,unique_post_authors,unique_comment_authors,unique_domains,bot_posts,bot_comments,deleted_comments
books,111052,105892,5160,2022-10-01 00:00:43,2022-10-30 23:59:48,29 days 23:59:05,50461,3736,48126,261,105892,2842,4665
conspiracy,455437,444001,11436,2022-10-01 00:00:02,2022-10-30 23:59:58,29 days 23:59:56,46447,4775,45063,833,444001,18931,20209
crypto,476948,462106,14842,2022-10-01 00:00:04,2022-10-30 23:59:50,29 days 23:59:46,34443,6530,30343,1270,462106,16628,26217
thedonald,1577,1083,494,2019-05-01 01:16:01,2019-06-29 23:57:26,59 days 22:41:25,860,288,633,54,1083,397,50


In [30]:
info_df.to_csv('dataset_summaries.csv')

In [9]:
# check no missing thread ids or timestamps

for key in datasets:
    original_length = len(datasets[key].data)
    new_length = len(datasets[key].data.dropna(axis=0, how='any', subset=['id', 'timestamp']))
    print(f'{key} nan rows:{original_length - new_length}')

books nan rows:0
conspiracy nan rows:0
crypto nan rows:0
thedonald nan rows:0


In [10]:
# check for removed or deleted comments
removed_comments = {}
removed_posts = {}
for key in datasets:
    comments = datasets[key].comments
    removed_comments[key] = comments[((comments.body=='[removed]') | (comments.body=='[deleted]'))]
    posts = datasets[key].posts
    removed_posts[key] = posts[((posts.body=='[removed]')|(posts.body=='[deleted]')|(posts.subject=='[removed]')|(posts.subject=='[deleted]'))]


In [26]:
print('dataset, #removed comments, #unique authors')
removed_comment_authors = {}
for key in removed_comments:
    print(f"{key}, {len(removed_comments[key])}, {len(removed_comments[key].author.unique())}")
    removed_comment_authors[key] = removed_comments[key].author.unique()

dataset, #removed comments, #unique authors
books, 2035, 1
conspiracy, 19468, 3
crypto, 21973, 3
thedonald, 31, 1


In [27]:
removed_comment_authors

{'books': array(['dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538'], dtype=object),
 'conspiracy': array(['dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538',
        '13cd89f4cb9cd1c5e6a28d9fb640e8454ae1e431c500b55c',
        '31c5b4906df83f326a568884e36aead0890f2c39a2f8b23c'], dtype=object),
 'crypto': array(['dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538',
        '67eda279a24a6d028efd7ac1f625bb2878863bdaab08b7d4',
        '2e77110dcd120527111b61f3535ad03750703b1f3c21b662'], dtype=object),
 'thedonald': array(['dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538'], dtype=object)}

In [47]:
for key in removed_comments:
    df = removed_comments[key]
    print(f'{key}: {len(df[df.author == "dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538"])}')

books: 2035
conspiracy: 19466
crypto: 21971
thedonald: 31


In [40]:
other_removed_comments = {}
for key in removed_comment_authors:
    if len(removed_comment_authors[key]) > 1:
        condition = (removed_comments[key].author == removed_comment_authors[key][1])
        if len(removed_comment_authors[key]) > 2:
            for i in range(2, len(removed_comment_authors[key])):
                condition = condition | (removed_comments[key].author == removed_comment_authors[key][i])

        other_removed_comments[key] = removed_comments[key][condition]

In [42]:
other_removed_comments['conspiracy']

,thread_id,id,timestamp,body,subject,author,image_file,domain,url,image_md5,subreddit,parent,score,date
174149,y264m6,is1icsp,2022-10-12 16:54:03,[deleted],NaN,13cd89f4cb9cd1c5e6a28d9fb640e8454ae1e431c500b55c,NaN,NaN,NaN,NaN,conspiracy,t1_is14gnm,1,2022-10-12
224904,y4sorq,isgdxry,2022-10-15 19:55:40,[removed],NaN,31c5b4906df83f326a568884e36aead0890f2c39a2f8b23c,NaN,NaN,NaN,NaN,conspiracy,t3_y4sorq,1,2022-10-15


In [43]:
other_removed_comments['crypto']

,thread_id,id,timestamp,body,subject,author,image_file,domain,url,image_md5,subreddit,parent,score,date
278770,y6x0of,isrpew7,2022-10-18 05:20:29,[removed],NaN,67eda279a24a6d028efd7ac1f625bb2878863bdaab08b7d4,NaN,NaN,NaN,NaN,CryptoCurrency,t1_isrm7k4,1,2022-10-18
338680,ya0im6,it8v1uq,2022-10-21 19:53:22,[deleted],NaN,2e77110dcd120527111b61f3535ad03750703b1f3c21b662,NaN,NaN,NaN,NaN,CryptoCurrency,t1_it8hpox,1,2022-10-21


In [22]:
author_counts = {}
for key in datasets:
    author_counts[key] = datasets[key].group_by_col('author')
    author_counts[key].sort_values('counts', ascending=False)

In [29]:
for key in author_counts:
    display(author_counts[key].sort_values('counts', ascending=False).head(5).style.set_caption(f"r/{key}: top authors"))

,author,counts
26827,883c847acfb7c40b18d504b206c5061c4454be34aa81c1bd,2215
43451,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,2047
18981,5fe11acea1ba780086614f6b6f5a36a449cc0902eb0306d1,650
12926,419d47aa6bb1b54248959880a65b4244c9ec86d938e78240,509
26610,872a7a2986dad903832622afd5519b99fafd319dd9f90d10,372


,author,counts
40186,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,19507
24799,883c847acfb7c40b18d504b206c5061c4454be34aa81c1bd,11972
29722,a3456a2aab42422d595e7bbd2ce62b81e9c8c8acc5d4bd5e,2120
41667,e4f5070a50730bf7c926f7841dd4f1b3bf9909030f3e858e,1913
20746,71d4d15d1a3ab0c83d2d6e0dd66642aa6fb93bc0945496b4,1827


,author,counts
29854,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,21995
18499,883c847acfb7c40b18d504b206c5061c4454be34aa81c1bd,6780
24991,b834f43c0cc7d1694ff9cc03c7e27e8d941cf8b4d0b9ebb1,4994
24745,b681b233b3fce366a02ac8c11f6cdf643f31989f8a6ad5b2,3776
7031,3463920785ebc2670f3c4524f9339dcc52059711cd085195,3272


,author,counts
73,11640fec8a8f4e5b056bae0d7978339da0b82986e847cb03,121
405,765b18cce2d0715a8697cb70cf1731462f61fb90ed761b7d,48
348,64cc625d1eeac4c69f9a4fb26ef19c42600a87225a0c083c,43
751,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,34
101,1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,18


In [59]:
author_name = '1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e'
#keys = ['crypto']
keys = datasets.keys()
for key in keys:
    print(f"r/{key}: {author_name} activity")
    display(datasets[key].data[datasets[key].data.author == author_name])

r/books: 1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e activity


,thread_id,id,timestamp,body,subject,author,image_file,domain,url,image_md5,subreddit,parent,score,date


r/conspiracy: 1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e activity


,thread_id,id,timestamp,body,subject,author,image_file,domain,url,image_md5,subreddit,parent,score,date


r/crypto: 1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e activity


,thread_id,id,timestamp,body,subject,author,image_file,domain,url,image_md5,subreddit,parent,score,date


r/thedonald: 1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e activity


,thread_id,id,timestamp,body,subject,author,image_file,domain,url,image_md5,subreddit,parent,score,date
117,bmmn3x,bmmn3x,2019-05-09 17:31:38,NaN,When The One True Donald's sub is the only one...,1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,https://i.imgur.com/YSaPSQy.png,i.imgur.com,https://i.imgur.com/YSaPSQy.png,NaN,thedonald,NaN,330,2019-05-09
141,boipow,boipow,2019-05-14 13:08:29,NaN,"I found this on the internet, thought you folk...",1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,https://i.imgur.com/lfgowkI.jpg,i.imgur.com,https://i.imgur.com/lfgowkI.jpg,NaN,thedonald,NaN,5,2019-05-14
155,bpsjn5,bpsjn5,2019-05-17 15:16:57,NaN,Patriot: a person who vigorously supports thei...,1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,https://i.imgur.com/GFFWq7o.jpg,i.imgur.com,https://i.imgur.com/GFFWq7o.jpg,NaN,thedonald,NaN,224,2019-05-17
170,br2qju,br2qju,2019-05-20 23:05:32,NaN,Donald vs Childish Gambino - Who would win in ...,1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,https://i.imgur.com/MK7yEZa.jpg,i.imgur.com,https://i.imgur.com/MK7yEZa.jpg,NaN,thedonald,NaN,50,2019-05-20
275,buuycg,buuycg,2019-05-30 16:05:06,NaN,The Most Iconic Donald,1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,https://i.imgur.com/8fGNMpk.jpg,i.imgur.com,https://i.imgur.com/8fGNMpk.jpg,NaN,thedonald,NaN,181,2019-05-30
418,bmmn3x,emxrol5,2019-05-09 17:32:03,https://www.reddit.com/r/SubredditDrama/commen...,NaN,1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,NaN,NaN,NaN,NaN,thedonald,t3_bmmn3x,46,2019-05-09
424,bmmn3x,emy4mgb,2019-05-09 19:30:49,"I heard that ""The Cyber"" is the Secret Service...",NaN,1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,NaN,NaN,NaN,NaN,thedonald,t1_emxvmgo,14,2019-05-09
490,boais5,enf1sar,2019-05-14 04:28:20,I'll have what he's having.,NaN,1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,NaN,NaN,NaN,NaN,thedonald,t3_boais5,5,2019-05-14
517,bpsjn5,enx3t61,2019-05-17 16:07:16,That's leaning toward Nationalist: a person wh...,NaN,1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,NaN,NaN,NaN,NaN,thedonald,t1_enx361b,16,2019-05-17
520,bpsjn5,enxiluz,2019-05-17 17:59:59,You have a lot more faith in humanity than I d...,NaN,1d917797acdb7a02eda4b0e6706c139b120a0d47f8dbc34e,NaN,NaN,NaN,NaN,thedonald,t1_enx9tp2,11,2019-05-17


In [60]:
for key in ['books']:
    display(author_counts[key].sort_values('counts', ascending=False).head(10).style.set_caption(f"r/{key}: top authors"))

,author,counts
26827,883c847acfb7c40b18d504b206c5061c4454be34aa81c1bd,2215
43451,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,2047
18981,5fe11acea1ba780086614f6b6f5a36a449cc0902eb0306d1,650
12926,419d47aa6bb1b54248959880a65b4244c9ec86d938e78240,509
26610,872a7a2986dad903832622afd5519b99fafd319dd9f90d10,372
16835,55455f7a32a4fac0aee65c28baf37885bea379e0f4b66bbc,166
23793,789b559632fb90a6089f92098bb89cc1e8ff7f706d70d85f,152
4926,19718d378f2d5b54d5cecd9a71f870108861d4937ea06814,150
19864,6460ae8dd20c14ff0a67ff44e083ea7359055a6ff6837fa4,140
42674,d8af18bd1733c743403c41986d3744f749196158a24d108c,113


In [68]:
author_name = 'd8af18bd1733c743403c41986d3744f749196158a24d108c'
keys = ['books']
#keys = datasets.keys()
for key in keys:
    print(f"r/{key}: {author_name} activity")
    display(datasets[key].data[datasets[key].data.author == author_name])

r/books: d8af18bd1733c743403c41986d3744f749196158a24d108c activity


,thread_id,id,timestamp,body,subject,author,image_file,domain,url,image_md5,subreddit,parent,score,date
5616,xsjglc,iql5vbd,2022-10-01 04:11:21,*Twenty Thousand Leagues Under the Sea* by Jul...,NaN,d8af18bd1733c743403c41986d3744f749196158a24d108c,NaN,NaN,NaN,NaN,books,t3_xsjglc,1,2022-10-01
5790,xsnhk2,iqlgyf5,2022-10-01 06:22:34,"Yes, manga and graphic novels are super popula...",NaN,d8af18bd1733c743403c41986d3744f749196158a24d108c,NaN,NaN,NaN,NaN,books,t3_xsnhk2,1,2022-10-01
5795,xsnypq,iqlhdhg,2022-10-01 06:28:05,Sounds like you and Helene Hanff would get alo...,NaN,d8af18bd1733c743403c41986d3744f749196158a24d108c,NaN,NaN,NaN,NaN,books,t3_xsnypq,1,2022-10-01
7768,xtfv1v,iqq0acc,2022-10-02 07:06:58,"Just FYI, that movie is *heavily* fictionalize...",NaN,d8af18bd1733c743403c41986d3744f749196158a24d108c,NaN,NaN,NaN,NaN,books,t1_iqpsdyr,1,2022-10-02
7770,xtfv1v,iqq0lna,2022-10-02 07:10:55,Great analysis! Hill House is my favorite Jack...,NaN,d8af18bd1733c743403c41986d3744f749196158a24d108c,NaN,NaN,NaN,NaN,books,t3_xtfv1v,1,2022-10-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96191,yfd14i,iu39idr,2022-10-28 06:04:50,She's presumably an English teacher with an En...,NaN,d8af18bd1733c743403c41986d3744f749196158a24d108c,NaN,NaN,NaN,NaN,books,t1_iu2z701,1,2022-10-28
96304,yf9lxi,iu3d052,2022-10-28 06:50:40,Reddit user u/ZeMastor put together [this webs...,NaN,d8af18bd1733c743403c41986d3744f749196158a24d108c,NaN,NaN,NaN,NaN,books,t3_yf9lxi,1,2022-10-28
96570,yfhwg3,iu3kjwl,2022-10-28 08:38:38,They're unrelated.\nThere are a lot of terribl...,NaN,d8af18bd1733c743403c41986d3744f749196158a24d108c,NaN,NaN,NaN,NaN,books,t3_yfhwg3,1,2022-10-28
104496,ygtvpt,iuarnzi,2022-10-29 22:01:05,"Yep, I mainly use it for the reviewing aspect ...",NaN,d8af18bd1733c743403c41986d3744f749196158a24d108c,NaN,NaN,NaN,NaN,books,t1_iuaguuc,1,2022-10-29


Who I've settled on removing in authors_to_remove.csv

In [70]:
to_remove = pd.read_csv('authors_to_remove.csv')

In [81]:
to_remove

,author,to_remove,note,books,conspiracy,crypto,thedonald
0,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,all,dummy author label for removed comments,1,1,1,1
1,883c847acfb7c40b18d504b206c5061c4454be34aa81c1bd,comments,moderator bot but also starts regular discussi...,1,1,1,0
2,5fe11acea1ba780086614f6b6f5a36a449cc0902eb0306d1,all,moderator for r/books,1,0,0,0
3,419d47aa6bb1b54248959880a65b4244c9ec86d938e78240,all,moderator for r/books,1,0,0,0
4,872a7a2986dad903832622afd5519b99fafd319dd9f90d10,all,spam bot in r/books,1,0,0,0
5,b681b233b3fce366a02ac8c11f6cdf643f31989f8a6ad5b2,all,'moderator bot in r/crypto only comments',0,0,1,0


In [98]:
datasets = pickle.load(open('datasets.p', 'rb'))

In [99]:
for key in datasets:
    remove_from_dataset = to_remove[to_remove[key] == 1]
    comments_to_remove = remove_from_dataset[remove_from_dataset.to_remove == 'comments'].author
    all_to_remove = remove_from_dataset[remove_from_dataset.to_remove == 'all'].author
    
    dataset = datasets[key]

    all_condition = (dataset.data.author.isin(all_to_remove))
    

    dataset.removed = dataset.data[all_condition]

    if len(comments_to_remove) > 0:
        comment_condition = (dataset.comments.author.isin(comments_to_remove))
        dataset.removed = pd.concat(
                (
                    dataset.removed,
                    dataset.comments[comment_condition]
                )
            )

    dataset.data = dataset.data[~dataset.data.id.isin(dataset.removed.id)]
    dataset.update_comments_posts()

In [100]:
datasets['books'].data

,thread_id,id,timestamp,body,subject,author,image_file,domain,url,image_md5,subreddit,parent,score,date
0,xsgu8l,xsgu8l,2022-10-01 00:11:11,NaN,"Neil Gaiman, Cory Doctorow And Other Authors P...",f44d5bd8146ae4baecd32037a8285c4f99690e85fa022ec5,NaN,deadline.com,https://deadline.com/2022/09/authors-open-lett...,NaN,books,NaN,1,2022-10-01
1,xsh6qv,xsh6qv,2022-10-01 00:27:38,NaN,"The New Person, a review by Shelley",703bd9ed6d2b178437b437b99074449db8137f1d00e6f7be,NaN,thebookreviewcrew.com,https://thebookreviewcrew.com/the-new-person-a...,NaN,books,NaN,1,2022-10-01
2,xshdvl,xshdvl,2022-10-01 00:36:53,NaN,Prescience 1: In Which I Talk About A Specific...,d9cacaec67989837c343381be2e4d5016c7743d419a34e84,NaN,story-arc-blog.com,https://story-arc-blog.com/2022/09/30/prescien...,NaN,books,NaN,1,2022-10-01
3,xsi14p,xsi14p,2022-10-01 01:07:47,[removed],Tracking Upcoming Releases,c466b9998753e56da1c6050ead680f8d2120eb91e2ef0fb9,NaN,self.books,https://www.reddit.com/r/books/comments/xsi14p...,NaN,books,NaN,1,2022-10-01
4,xsis1h,xsis1h,2022-10-01 01:44:22,*Spoilers below*\n&#x200B;\nOkay so I started ...,"Kiera Cass' third book, The One, is by far the...",920f0705acd330eaf28719163e0801e1155d19b4a213351a,NaN,self.books,https://www.reddit.com/r/books/comments/xsis1h...,NaN,books,NaN,1,2022-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111046,yhks0d,iufrknp,2022-10-30 23:58:32,I'm just scrolling through this thread looking...,NaN,c1487bafa8953938eee4672e4d97aed3997d2905ed77e871,NaN,NaN,NaN,NaN,books,t1_iuede6w,1,2022-10-30
111047,yhf64b,iufrm4t,2022-10-30 23:58:52,Seriously by about book 10 I was hoping the wh...,NaN,4a2ae05277cb0aa85a64fdec8358f8f628b8f48d137d1944,NaN,NaN,NaN,NaN,books,t1_iue83in,1,2022-10-30
111049,yeibp7,iufroik,2022-10-30 23:59:23,"Yes, this is true, but students won’t read it ...",NaN,00b26decc84c6c93f88dc30f80178822d4c1bbd4022d4e5e,NaN,NaN,NaN,NaN,books,t1_ityf6e7,1,2022-10-30
111050,yhks0d,iufrp7w,2022-10-30 23:59:33,"Yeah, I was gonna say, sometimes that's why I ...",NaN,95437de983251f9f21c0dc6da4c8298daffb9677ecd86566,NaN,NaN,NaN,NaN,books,t1_iuede6w,1,2022-10-30


In [103]:
datasets['books'].removed[datasets['books'].removed.author == 'dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538']

,thread_id,id,timestamp,body,subject,author,image_file,domain,url,image_md5,subreddit,parent,score,date
32,xsq7ba,xsq7ba,2022-10-01 08:41:05,NaN,The aeneid,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,NaN,NaN,NaN,NaN,books,NaN,1,2022-10-01
260,xtwp69,xtwp69,2022-10-02 18:58:02,NaN,Do You Have a Loved Pet That You'd Like to Tak...,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,NaN,NaN,NaN,NaN,books,NaN,1,2022-10-02
495,xv5tii,xv5tii,2022-10-04 04:53:42,NaN,"Jayme Ringleb dove into his debut collection, ...",dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,NaN,NaN,NaN,NaN,books,NaN,1,2022-10-04
798,xwu6hl,xwu6hl,2022-10-06 02:52:01,NaN,The Art of the Deal - Book Review,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,NaN,NaN,NaN,NaN,books,NaN,1,2022-10-06
1013,xy3q0b,xy3q0b,2022-10-07 16:35:43,NaN,What book character was ruined in the movie ve...,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,NaN,NaN,NaN,NaN,books,NaN,1,2022-10-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110617,yhebqr,iufdqh6,2022-10-30 22:17:47,[deleted],NaN,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,NaN,NaN,NaN,NaN,books,t1_iuf9gxy,1,2022-10-30
110623,yhks0d,iufdu08,2022-10-30 22:18:29,[removed],NaN,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,NaN,NaN,NaN,NaN,books,t3_yhks0d,1,2022-10-30
110647,yhks0d,iufenzx,2022-10-30 22:24:26,[deleted],NaN,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,NaN,NaN,NaN,NaN,books,t1_iueudho,1,2022-10-30
110990,yhks0d,iufph60,2022-10-30 23:42:28,[removed],NaN,dc98995ab1ecc1f812b1cacbe0942f2da34fc4639168e538,NaN,NaN,NaN,NaN,books,t1_iufklsw,1,2022-10-30


In [105]:
pickle.dump(datasets, open('cleaned_datasets.p', 'wb'))